# Diferentiall express analysis

In [1]:
import scanpy as sc
import decoupler as dc

# Only needed for processing
import numpy as np
import pandas as pd
from anndata import AnnData
from pydeseq2.dds import DeseqDataSet, DefaultInference
from pydeseq2.ds import DeseqStats


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
experiment = 'RNAseq_abundances_adjusted_combat_inmose'

comparison = 'young.vs.old'

In [3]:
adata = pd.read_csv(f'/home/amore/work/data/{experiment}_gene_symbol_expression.csv', index_col=0)
adata

,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,...,C4orf36.1,TUSC2P1,Unnamed: 34327,OR4M2-OT1,H2BK1,OR1Q1BP,Unnamed: 34331,Unnamed: 34332,TBCEL-TECTA,Age
Sample,,,,,,,,,,,,,,,,,,,,,
SRR13758984,21281762,0,15759712,0,11295483,0,35294310,0,18333093,0,...,12741169,0,0,0,0,0,0,2773,0,91.0
SRR13758985,8542646,0,15035432,0,0,0,26637602,0,1389125,15716930,...,10663212,0,0,0,0,0,0,1328,0,86.0
SRR13758986,4079945,0,5410942,0,5658191,0,10946301,4491317,7747412,9046855,...,4248288,0,0,0,0,0,0,1,0,69.0
SRR13758987,14758557,0,12809471,0,0,0,24093106,0,11562944,16086208,...,10497182,0,0,0,0,0,0,1614,0,83.0
SRR13758988,3623061,0,15529838,23957067,0,0,25492722,10773730,12889518,0,...,10506926,0,0,0,0,0,0,795,0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR1555210,37402439,0,21355420,1,1212397,1,87213838,3370216,5884457,951798,...,23486240,0,0,0,0,0,0,0,5351828,27.5
SRR1555211,48027788,0,89024445,1348907,24399,1,148387777,3685378,113719675,2233983,...,34582470,0,0,0,1956002,0,0,0,0,27.5
SRR1555212,84899250,0,167383416,407713,37284600,1,219777948,58357903,79405469,147215871,...,81916604,0,1537700,0,824055,0,1040940,12712840,5952181,27.5


In [4]:
age_series = adata['Age']
adata

,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,...,C4orf36.1,TUSC2P1,Unnamed: 34327,OR4M2-OT1,H2BK1,OR1Q1BP,Unnamed: 34331,Unnamed: 34332,TBCEL-TECTA,Age
Sample,,,,,,,,,,,,,,,,,,,,,
SRR13758984,21281762,0,15759712,0,11295483,0,35294310,0,18333093,0,...,12741169,0,0,0,0,0,0,2773,0,91.0
SRR13758985,8542646,0,15035432,0,0,0,26637602,0,1389125,15716930,...,10663212,0,0,0,0,0,0,1328,0,86.0
SRR13758986,4079945,0,5410942,0,5658191,0,10946301,4491317,7747412,9046855,...,4248288,0,0,0,0,0,0,1,0,69.0
SRR13758987,14758557,0,12809471,0,0,0,24093106,0,11562944,16086208,...,10497182,0,0,0,0,0,0,1614,0,83.0
SRR13758988,3623061,0,15529838,23957067,0,0,25492722,10773730,12889518,0,...,10506926,0,0,0,0,0,0,795,0,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR1555210,37402439,0,21355420,1,1212397,1,87213838,3370216,5884457,951798,...,23486240,0,0,0,0,0,0,0,5351828,27.5
SRR1555211,48027788,0,89024445,1348907,24399,1,148387777,3685378,113719675,2233983,...,34582470,0,0,0,1956002,0,0,0,0,27.5
SRR1555212,84899250,0,167383416,407713,37284600,1,219777948,58357903,79405469,147215871,...,81916604,0,1537700,0,824055,0,1040940,12712840,5952181,27.5


In [5]:
adata=adata.astype(int)
adata.isnull().sum().any()

False

In [6]:
ages=adata['Age']
adata=adata.drop('Age',axis=1)
andata=AnnData(adata.to_numpy(), obs=pd.DataFrame(ages))
andata.obs_names = adata.index
andata.var_names = adata.columns


In [7]:
adata = andata #AnnData(adata.to_numpy(), dtype=np.int32)
adata.var_names_make_unique()
adata

AnnData object with n_obs × n_vars = 270 × 34333
    obs: 'Age'

In [8]:
# Process treatment information
adata.obs['condition'] = age_series.apply(lambda age: 'young' if age < 35 else ('old' if age >= 65 else 'middle'))


In [9]:
# Obtain genes that pass the thresholds
genes = dc.filter_by_expr(adata, group='condition', min_count=10, min_total_count=15, large_n=1, min_prop=1)

# Filter by these genes
adata = adata[:, genes].copy()
adata

AnnData object with n_obs × n_vars = 270 × 25378
    obs: 'Age', 'condition'

In [10]:
# Build DESeq2 object
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    adata=adata,
    design_factors='condition',
    refit_cooks=True,
    inference=inference,
)


In [11]:
dds.deseq2()

Fitting size factors...
... done in 0.12 seconds.

Fitting dispersions...
... done in 21.74 seconds.

Fitting dispersion trend curve...
... done in 0.60 seconds.

Fitting MAP dispersions...
... done in 24.20 seconds.

Fitting LFCs...
... done in 31.64 seconds.

Replacing 6537 outlier genes.

Fitting dispersions...
... done in 6.85 seconds.

Fitting MAP dispersions...
... done in 7.04 seconds.

Fitting LFCs...
... done in 11.06 seconds.



In [12]:
def get_DDS(younger_group, older_group, save=True):
    comparison = f'{younger_group}.vs.{older_group}'
    stat_res = DeseqStats(
        dds,
        contrast=["condition", younger_group, older_group],
        inference=inference
    )
    stat_res.summary()
    results_df = stat_res.results_df
    if result:
        results_df.to_csv(f'/home/amore/work/data/{experiment}_{comparison}_DDS.csv', header=True)
    return result_df